In [1]:
import pandas as pd
import requests

# Fetch the raw WITHDRAWAL data by category/department for each fiscal year and quarter from FY2013 to FY2024
# This script allows for adjustment of the fiscal years and quarters as needed
# Draws the data from the Fiscal Data API provided by the U.S. Department of the Treasury for its Daily Statement (DTS)

# Define base URL
url = "https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v1/accounting/dts/deposits_withdrawals_operating_cash"

# Create empty list to store data
all_data = []

# Loop through fiscal years and quarters to solve for pagination limits within the API
for year in range(2013, 2025):  # Adjust years as needed
    for quarter in range(1, 5):  # Quarters 1 to 4
        print(f"Fetching data for FY {year}, Quarter {quarter}...")
        
        # Define the parameters for each year and quarter; we're limiting to withdrawals only
        # Many other parameters can be added as needed; consult the API documentation for more options
        params = {
            "filter": f"transaction_type:eq:Withdrawals,record_fiscal_year:eq:{year},record_fiscal_quarter:eq:{quarter}",
            "page[size]": 10000
        }
        
        # Make the GET request
        response = requests.get(url, params=params)
        
        # See response status
        if response.status_code == 200:
            data = response.json()  # Parse the JSON response
            
            # Extract the 'data' field and append it to the list
            if "data" in data:
                all_data.extend(data["data"])
            else:
                print(f"No data found for FY {year}, Quarter {quarter}.")
        else:
            print(f"Request failed for FY {year}, Quarter {quarter} with status code {response.status_code}: {response.text}")

# Convert the list to DataFrame
withdrawals_historic = pd.DataFrame(all_data)

# Convert record_date to datetime
if not withdrawals_historic.empty:
    withdrawals_historic["record_date"] = pd.to_datetime(withdrawals_historic["record_date"])

# Display DataFrame info
withdrawals_historic.info()

# Print the range of dates in the data
if not withdrawals_historic.empty:
    print(f"Max date in API data: {withdrawals_historic['record_date'].max()}")
    print(f"Min date in API data: {withdrawals_historic['record_date'].min()}")

# Display the first few rows
withdrawals_historic.head()



Fetching data for FY 2013, Quarter 1...
Fetching data for FY 2013, Quarter 2...
Fetching data for FY 2013, Quarter 3...
Fetching data for FY 2013, Quarter 4...
Fetching data for FY 2014, Quarter 1...
Fetching data for FY 2014, Quarter 2...
Fetching data for FY 2014, Quarter 3...
Fetching data for FY 2014, Quarter 4...
Fetching data for FY 2015, Quarter 1...
Fetching data for FY 2015, Quarter 2...
Fetching data for FY 2015, Quarter 3...
Fetching data for FY 2015, Quarter 4...
Fetching data for FY 2016, Quarter 1...
Fetching data for FY 2016, Quarter 2...
Fetching data for FY 2016, Quarter 3...
Fetching data for FY 2016, Quarter 4...
Fetching data for FY 2017, Quarter 1...
Fetching data for FY 2017, Quarter 2...
Fetching data for FY 2017, Quarter 3...
Fetching data for FY 2017, Quarter 4...
Fetching data for FY 2018, Quarter 1...
Fetching data for FY 2018, Quarter 2...
Fetching data for FY 2018, Quarter 3...
Fetching data for FY 2018, Quarter 4...
Fetching data for FY 2019, Quarter 1...


,record_date,account_type,transaction_type,transaction_catg,transaction_catg_desc,transaction_today_amt,transaction_mtd_amt,transaction_fytd_amt,table_nbr,table_nm,src_line_nbr,record_fiscal_year,record_fiscal_quarter,record_calendar_year,record_calendar_quarter,record_calendar_month,record_calendar_day
0,2012-10-01,Federal Reserve Account,Withdrawals,Commodity Credit Corporation programs,null,39,39,39,II,Deposits and Withdrawals of Operating Cash,33,2013,1,2012,4,10,01
1,2012-10-01,Federal Reserve Account,Withdrawals,Defense Vendor Payments (EFT),null,943,943,943,II,Deposits and Withdrawals of Operating Cash,34,2013,1,2012,4,10,01
2,2012-10-01,Federal Reserve Account,Withdrawals,Education Department programs,null,739,739,739,II,Deposits and Withdrawals of Operating Cash,35,2013,1,2012,4,10,01
3,2012-10-01,Federal Reserve Account,Withdrawals,Energy Department programs,null,160,160,160,II,Deposits and Withdrawals of Operating Cash,36,2013,1,2012,4,10,01
4,2012-10-01,Federal Reserve Account,Withdrawals,Federal Employees Insurance Payments,null,161,161,161,II,Deposits and Withdrawals of Operating Cash,37,2013,1,2012,4,10,01


In [6]:

# Save the DataFrame to a CSV file for backup in case of API changes or data loss/removal by government
withdrawals_historic.to_csv("output/withdrawals_historic.csv", index=False)
# Save the DataFrame to a pickle file for faster reloading with formatting for adding daily statements via automated pipeline
withdrawals_historic.to_pickle("data/withdrawals_historic.pkl")
withdrawals_historic.to_parquet("data/withdrawals_historic.parquet")

